# HW2 流程圖和學習歷程

建立一個債券YTM、spot rate、forward rate的試算器並且直接在執行畫面中輸出，可重複執行不用再開啟

In [ ]:
while True:
    print("This is a calculator of yield to maturity, spot rate, and forward rate.")
    choice=int(input("If you'd like to calculate yield to maturity of one bond, then please enter 1.\nIf you'd like to calculate spot rate generated by one zero coupon bond, then please enter 2.\nIf you'd like to calculate forward rate based on a set of bonds, then please enter 3.\nNow you can enter. "))
#使用者可以選擇要計算YTM(則輸入1)、spot rate(則輸入2)或者forward rate(則輸入3)的對照表

如果使用者輸入1，則進入YTM計算區

In [ ]:
     if choice==1:
        current_bond_price=float(input("Please enter the cuurent bond price: "))
        par_value=float(input("Please enter the par value of the bond: "))
        coupon_rate=float(input("Please enter the coupon rate(%): "))
        year_to_maturity=float(input("Please enter year to maturity of the bond: "))
        proceed_payment_method=int(input("If proceeds are paid annually, please enter 1.\nIf proceeds are paid semiannually, please enter 2.\nIf proceeds are paid quarterly, please enter 4.\n Now please enter: "))
#使用者依照指示分別輸入該檔債券的現在市場價格、票面價值、年化利率、債券到期時間、以及利息支付方式(一年計息、半年計息、一季一季計息)

In [ ]:
        coupon=par_value*coupon_rate/100/proceed_payment_method
        total_payment_times=year_to_maturity*proceed_payment_method
#為方便後續計算，將年化利率，轉成每期計算利率，並以小數表示

由於用sympy去解ytm計算方程式(通常是ytm的高次方程式)效率太差，以下練習使用十分逼近法，逼近ytm，逼近至小數點下五位

In [ ]:
        ytm=0
        for a in range(0,10,1):
            a=a/10
            time=1
            present_value=0
            while time<=total_payment_times-1:
                present_value+=coupon/(1+(a/proceed_payment_method))**time
                time+=1
            present_value+=(coupon+par_value)/(1+(a/proceed_payment_method))**total_payment_times
            if present_value<current_bond_price:
                ytm=a-0.1
                break
            else:
                continue
        for a in range(int(ytm*100),int((ytm+0.1)*100)+1,1):
            a=a/100
            time=1
            present_value=0
            while time<=total_payment_times-1:
                present_value+=coupon/(1+(a/proceed_payment_method))**time
                time+=1
            present_value+=(coupon+par_value)/(1+(a/proceed_payment_method))**total_payment_times
            if present_value<current_bond_price:
                ytm=a-0.01
                break
            else:
                continue
        for a in range(int(ytm*1000),int((ytm+0.01)*1000)+1,1):
            a=a/1000
            time=1
            present_value=0
            while time<=total_payment_times-1:
                present_value+=coupon/(1+(a/proceed_payment_method))**time
                time+=1
            present_value+=(coupon+par_value)/(1+(a/proceed_payment_method))**total_payment_times
            if present_value<current_bond_price:
                ytm=a-0.001
                break
            else:
                continue
        for a in range(int(ytm*10000),int((ytm+0.001)*10000)+1,1):
            a=a/10000
            time=1
            present_value=0
            while time<=total_payment_times-1:
                present_value+=coupon/(1+(a/proceed_payment_method))**time
                time+=1
            present_value+=(coupon+par_value)/(1+(a/proceed_payment_method))**total_payment_times
            if present_value<current_bond_price:
                ytm=a-0.0001
                break
            else:
                continue
        for a in range(int(ytm*100000),int((ytm+0.0001)*100000),1):
            a=a/100000
            time=1
            present_value=0
            while time<=total_payment_times-1:
                present_value+=coupon/(1+(a/proceed_payment_method))**time
                time+=1
            present_value+=(coupon+par_value)/(1+(a/proceed_payment_method))**total_payment_times
            if present_value<current_bond_price:
                ytm=a-0.00001
                break
            else:
                continue    
        YTM_percent=ytm*100
        print("Yield to maturity is "+str(YTM_percent)+"%")
#最後的ytm輸出形式轉化成百分率

如果使用者選擇2，則進入spot rate計算區，使用者會根據指示輸入零息票債券的資訊

In [ ]:
    elif choice==2:
        year_to_maturity=int(input("Please enter year to maturity of the zero coupon bond: "))
        current_bond_price=float(input("Please enter the price of unit coupon bond: "))

根據零息票券的資訊計算spot rate，解的方程式不複雜，因此十分逼近法可能就不需要使用

In [ ]:
        for a in range(0,100000+1,1):
            if 1/(1+(a/100000))**year_to_maturity<current_bond_price:
                spot_rate=round((a/100000-0.000001)*100,3)
                print(str(year_to_maturity)+" year spot rate of interest is "+str(spot_rate)+"%")
                break
            else:
                continue

如果使用者選擇3，則進入forward rate計算區，使用者會根據指示輸入一堆債券資料，以求取未來各期間的forward rate

這邊需另外說明。由於債券的次級市場在各國交易不發達(除了美國)，故forward rate或spot rate的計算應基於類似美國的市場環境做計算。美國政府發行treasury bill的頻率很高且交易量大，基於效率市場假說，市場上的債券價格通常不會出現偏差，因此只要取得市場上某一組流動性足夠的債券就可以找出未來各期間的forward rate。另外由於forward rate可以經由spot rate求出來，而欲求取n期spot rate則需要n檔債券去identify，以滿足唯一性。另外為避免使用數值內插的估計，因此此試算器會要求輸入一組天期為等差的債券。
舉例來說，如果使用者想要計算未來三年以半年為期間各期的遠期利率，由於這些遠期利率可以由六個spot rate轉換而成，所以使用者會被要求依序輸入6檔天期為等差的債券資料(在未來6個月、一年、一年半、兩年、兩年半、三年到期的債券)

In [ ]:
    elif choice==3:
        year_to_maturity=float(input("Please enter the number of years you want to know the variation of forward rate.\n"))
        period_duration=float(input("Please enter the duration of each period during which forward rate are calculated.(annually, 1, semiannually, 2, and quarterly, 4)\n"))
        number_of_bond_required=int(year_to_maturity*period_duration)
        print("To calculate forward rate for each period in the future, please enter the data of bonds in order of short to long year to maturity.")


以下使用while loop去將一組債券的資訊轉成天期為等差的零息票債券現值

In [ ]:
        bond_number=0
        spot_rate=[]
        while bond_number<=number_of_bond_required-1:
            bond_number+=1
            current_bond_price=float(input("Please enter the bond price: "))
            par_value=float(input("Please enter the par value of the bond: "))
            coupon_rate=float(input("Please enter the coupon rate(%): "))
            if period_duration==1:
                coupon_rate=coupon_rate/100/1
            elif period_duration==2:
                coupon_rate=coupon_rate/100/2
            elif period_duration==4:
                coupon_rate=coupon_rate/100/4
            if bond_number==1:
                f_01=current_bond_price/((coupon_rate+1)*par_value)
                spot_rate.append(f_01)
            elif bond_number>1:
                present_value=0
                for n in range(1, bond_number,1):
                    present_value+=coupon_rate*par_value*spot_rate[n-1]
                f=(current_bond_price-present_value)/((coupon_rate+1)*par_value)
                spot_rate.append(f)

以下就是以轉換過後得到天期為等差的零息票債券現值轉成spot rate，再換成forward rate，然後forward rate的表示形式是年化利率百分比

In [ ]:
        import sys
        from prettytable import PrettyTable
        spot_rate.insert(0,1)
        if period_duration==1:
            table= PrettyTable(["year\year",*range(0,number_of_bond_required+1,1)])
        elif period_duration==2:
            table= PrettyTable(["half-year\half-year",*range(0,number_of_bond_required+1,1)])
        elif period_duration==4:
            table= PrettyTable(["quarter\quarter",*range(0,number_of_bond_required+1,1)])
        for n in range(0,number_of_bond_required+1,1):
            row=[]
            row.append(n)
            time=0
            while time<n:
                row.append("-")
                time+=1
            time=0
            for k in spot_rate:
                if time==0:
                    row.append(1-1)
                    time=time+1
                elif time>0:
                    if period_duration==1:
                        row.append(round(((1/(k/spot_rate[0]))**(1/time)-1)*100,3))
                        time=time+1
                    elif period_duration==2:
                        row.append(round((((1/(k/spot_rate[0]))**(1/time)-1)*2)*100,3))
                        time=time+1
                    elif period_duration==4:
                        row.append(round((((1/(k/spot_rate[0]))**(1/time)-1)*4)*100,3))
                        time=time+1
            table.add_row(row)
            spot_rate.pop(0)
        print(table)